# Generate Random Models

In testing, we often need to generate random models. `schematics` offer this capability via a feature called [Model Mocking]



[Model Mocking]: https://schematics.readthedocs.io/en/latest/usage/models.html#model-mocking

In [1]:
import schematics

class EnvironmentMetadata(schematics.Model):
    name = schematics.types.StringType(required=True, max_length=30)
    display_name = schematics.types.StringType(serialized_name="displayName", max_length=255, serialize_when_none=False)
    description = schematics.types.StringType(max_length=255, serialize_when_none=False)

In [2]:
environment_metadata = EnvironmentMetadata.get_mock_object()

In [3]:
environment_metadata.name

'FFbWLmPl2lAYVC1QHL8mj59Ai87Ioh'

In [4]:
environment_metadata.display_name

'8Uc5vsvEXQcEZ1BPdL3LTxA7twNQEIqsrfF8cmR5hPJFZkvdMw7kQIjSvmmmgnN9RpULCeOYYOnSL0HlG2csVpFd9LLz8lNJmzoP3AaFk5DL1xn91dwDt1s75mwkVW4ysI4xp4xbGByeYDZ90pTUTX683pjJgU0F0cmscbChZBz8TgCobxuJ3wjD44aiuihvzfd6qmJu1Sw6hLO7fIatn6QMa4aEDg9banOdhD15oubhQnwp4DZkWgk'

In [5]:
environment_metadata.description

'YA9YQ1yRfGjgG0Rx57aggd2QjHvORapwAEOmbzgHviyUcI4epNfQ1kuL19itldURCeOy3rWvuWG4bCN1hNUd9clFr48ec2nmHfcBx5mqMZIEmFwxrrE56vBnAxzrt1yG0O1S7PB3CwYOw3Q468rqDgBcKZQpXst'

In [6]:
environment_metadata.to_primitive()

{'name': 'FFbWLmPl2lAYVC1QHL8mj59Ai87Ioh',
 'displayName': '8Uc5vsvEXQcEZ1BPdL3LTxA7twNQEIqsrfF8cmR5hPJFZkvdMw7kQIjSvmmmgnN9RpULCeOYYOnSL0HlG2csVpFd9LLz8lNJmzoP3AaFk5DL1xn91dwDt1s75mwkVW4ysI4xp4xbGByeYDZ90pTUTX683pjJgU0F0cmscbChZBz8TgCobxuJ3wjD44aiuihvzfd6qmJu1Sw6hLO7fIatn6QMa4aEDg9banOdhD15oubhQnwp4DZkWgk',
 'description': 'YA9YQ1yRfGjgG0Rx57aggd2QjHvORapwAEOmbzgHviyUcI4epNfQ1kuL19itldURCeOy3rWvuWG4bCN1hNUd9clFr48ec2nmHfcBx5mqMZIEmFwxrrE56vBnAxzrt1yG0O1S7PB3CwYOw3Q468rqDgBcKZQpXst'}

## Generate Nested Models

In [7]:
import json
import random

import schematics

class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
class Contact(schematics.Model):
    name = schematics.types.StringType(required=True)
    
    # Nested: A single object
    unix_user = schematics.types.ModelType(UnixUser, serialized_name="unixUser", required=True)
    
    # Nested: Many objects
    phones = schematics.types.ListType(schematics.types.ModelType(Phone), default=[])

In [8]:
for _ in range(3):
    contact = Contact.get_mock_object()
    print("-" * 72)
    print(json.dumps(contact.to_primitive(), indent=4))

------------------------------------------------------------------------
{
    "name": "6e3TqPnu8",
    "unixUser": {
        "alias": "ZJWXKS",
        "is_admin": false
    },
    "phones": []
}
------------------------------------------------------------------------
{
    "name": "vpGm38dG4",
    "unixUser": {
        "alias": "0IDpPYGTnMXm",
        "is_admin": true
    },
    "phones": [
        {
            "number": "xDVqIT",
            "kind": "home"
        },
        {
            "number": "ijlnPZ1fW",
            "kind": "home"
        },
        {
            "number": "HzoRJ4Di",
            "kind": "work"
        },
        {
            "number": "hznAq0XU9Dem6",
            "kind": "home"
        },
        {
            "number": "hSvD2SVpWee",
            "kind": "other"
        }
    ]
}
------------------------------------------------------------------------
{
    "name": "2O5Vygi8bouOcrY",
    "unixUser": {
        "alias": "Owhav",
        "is_admin": true
    

## Overriding the Random Generator

If we need to override some fields, we can create a dictionary and pass that into `get_mock_object()`:

In [9]:
contact = Contact.get_mock_object(overrides={"name": "Johan"})
print(json.dumps(contact.to_primitive(), indent=4))

{
    "name": "Johan",
    "unixUser": {
        "alias": "Cks1",
        "is_admin": false
    },
    "phones": [
        {
            "number": "SONpTuG7oiu6",
            "kind": "work"
        },
        {
            "number": "xYjxIsT",
            "kind": "other"
        },
        {
            "number": "dNLiwTUoh353Vz",
            "kind": "work"
        },
        {
            "number": "Ji",
            "kind": "mobile"
        },
        {
            "number": "FVUQs",
            "kind": "other"
        },
        {
            "number": "OzRpY",
            "kind": "work"
        },
        {
            "number": "ACU7",
            "kind": "mobile"
        },
        {
            "number": "XQ",
            "kind": "work"
        },
        {
            "number": "VxlVorYN",
            "kind": "home"
        },
        {
            "number": "SRtGLf1g8jAYR",
            "kind": "work"
        },
        {
            "number": "fpx4mHGKbC",
            "kind": "

## Caveats

Note that in the example above, the random models might not be valid. For example, the phone number field does not satisfy the regular expression:

In [10]:
phone = Phone.get_mock_object()
print(phone.to_primitive())
try:
    phone.validate()
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

{'number': 'HZn6R09C2hBK', 'kind': 'other'}
Error: {"number": ["String value did not match validation regex."]}


For this reason, if we want to generate our own valid models, we need to do it ourselves and do not rely on `get_mock_object`.

In [11]:
def generate_random_phone_number():
    number = "-".join(
        str("".join(random.sample("0123456789", n)))
        for n in [3, 3, 4]
    )
    return number

class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
    @classmethod
    def get_mock_object(cls, context=None, overrides=None):
        obj = super().get_mock_object(context=context, overrides=overrides or {})
        try:
            obj.validate()
        except schematics.exceptions.DataError:
            obj.number = generate_random_phone_number()
        return obj

In [12]:
for _ in range(5):
    phone = Phone.get_mock_object()
    print(phone.to_primitive())
    phone.validate()

{'number': '504-465-7160', 'kind': 'mobile'}
{'number': '208-436-9175', 'kind': 'mobile'}
{'number': '804-251-7528', 'kind': 'work'}
{'number': '048-619-9471', 'kind': 'work'}
{'number': '825-695-0345', 'kind': 'mobile'}


In testing, sometimes we need invalid random models, which means the above method will not satisfy. In which case it helps to keep the inherited `get_mock_object()` method untouch and create a separate generator.

Advantage of overriding the `get_mock_object()` with our own is we can use it to generate a valid nested model.

In [13]:
def generate_random_phone_number():
    number = "-".join(
        str("".join(random.sample("0123456789", n)))
        for n in [3, 3, 4]
    )
    return number

class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
    @classmethod
    def get_mock_object(cls, context=None, overrides=None):
        obj = super().get_mock_object(context=context, overrides=overrides or {})
        try:
            obj.validate()
        except schematics.exceptions.DataError:
            obj.number = generate_random_phone_number()
        return obj

class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
class Contact(schematics.Model):
    name = schematics.types.StringType(required=True)
    
    # Nested: A single object
    unix_user = schematics.types.ModelType(UnixUser, serialized_name="unixUser", required=True)
    
    # Nested: Many objects
    phones = schematics.types.ListType(schematics.types.ModelType(Phone), )
    
for _ in range(2):
    contact = Contact.get_mock_object()
    print("-" * 72)
    print(json.dumps(contact.to_primitive(), indent=4))
    contact.validate()

------------------------------------------------------------------------
{
    "name": "",
    "unixUser": {
        "alias": "kGeOu",
        "is_admin": false
    },
    "phones": [
        {
            "number": "418-478-6921",
            "kind": "home"
        },
        {
            "number": "473-071-1754",
            "kind": "mobile"
        },
        {
            "number": "428-530-7629",
            "kind": "work"
        },
        {
            "number": "197-451-2598",
            "kind": "other"
        },
        {
            "number": "346-850-4567",
            "kind": "work"
        },
        {
            "number": "296-347-1048",
            "kind": "other"
        },
        {
            "number": "260-901-5716",
            "kind": "home"
        },
        {
            "number": "160-709-0975",
            "kind": "work"
        }
    ]
}
------------------------------------------------------------------------
{
    "name": "LW",
    "unixUser": {
      

In [14]:
class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
    @classmethod
    def get_valid_mock_object(cls, context=None, overrides=None):
        obj = cls.get_mock_object(context=context, overrides=overrides or {})
        obj.number = generate_random_phone_number()
        return obj

In [15]:
for _ in range(5):
    phone = Phone.get_valid_mock_object()
    print(phone.to_primitive())
    phone.validate()

{'number': '520-462-7621', 'kind': 'other'}
{'number': '296-182-7953', 'kind': 'mobile'}
{'number': '685-937-9057', 'kind': 'home'}
{'number': '041-519-9812', 'kind': 'other'}
{'number': '082-251-0297', 'kind': 'other'}


## Summary

* Generate a random model with `Class.get_mock_object()`
* Override some field with the `get_mock_object(orverrides={some-dict})`
* Overriding `get_mock_object()` works in nested models
* The generated model might not be valid
* To ensure a valid model, write our own version of `get_mock_object()`